In [ ]:
# hide
%load_ext autoreload
%autoreload 2
%load_ext nb_black
%load_ext lab_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp download

<IPython.core.display.Javascript object>

# Download

> Downloaders for Numerai Classic and Numerai Signals.

In [ ]:
# hide
from nbdev.showdoc import *

<IPython.core.display.Javascript object>

In [ ]:
# export
import os
import glob
import json
import shutil
from rich.tree import Tree
from datetime import datetime
from rich.console import Console
from typeguard import typechecked
from pathlib import Path, PosixPath
from abc import ABC, abstractmethod
from rich import print as rich_print
from numerapi import NumerAPI, SignalsAPI
from dateutil.relativedelta import relativedelta, FR

from google.cloud import storage

<IPython.core.display.Javascript object>

## 0. BaseIO

There are common methods needed for `Downloaders` and `Submittors`. `BaseIO` implements this functionality and allows us to make abstract base classes. Namely, `BaseDownloader` and `BaseSubmittor`.

In [ ]:
# export
@typechecked
class BaseIO(ABC):
    """
    Basic functionality for IO like downloading and uploading.
    :param directory_path: Base folder for IO. Will be created if it does not exist.
    """

    def __init__(self, directory_path: str):
        self.dir = Path(directory_path)
        self._create_directory()

    def remove_base_directory(self):
        """Remove directory with all contents."""
        abs_path = self.dir.resolve()
        rich_print(
            f":warning: [red]Deleting directory for '{self.__class__.__name__}[/red]' :warning:\nPath: '{abs_path}'"
        )
        shutil.rmtree(abs_path)

    def download_file_from_gcs(self, bucket_name: str, gcs_path: str):
        """
        Get file from GCS bucket and download to local directory.
        :param gcs_path: Path to file on GCS bucket.
        """
        blob_path = str(self.dir.resolve())
        blob = self._get_gcs_blob(bucket_name=bucket_name, blob_path=blob_path)
        blob.download_to_filename(gcs_path)
        rich_print(
            f":cloud: :page_facing_up: Downloaded GCS object '{gcs_path}' from bucket '{blob.bucket.id}' to local directory '{blob_path}'. :page_facing_up: :cloud:"
        )

    def upload_file_to_gcs(self, bucket_name: str, gcs_path: str, local_path: str):
        """
        Upload file to some GCS bucket.
        :param gcs_path: Path to file on GCS bucket.
        """
        blob = self._get_gcs_blob(bucket_name=bucket_name, blob_path=gcs_path)
        blob.upload_from_filename(local_path)
        rich_print(
            f":cloud: :page_facing_up: Local file '{local_path}' uploaded to '{gcs_path}' in bucket {blob.bucket.id}:page_facing_up: :cloud:"
        )

    def download_directory_from_gcs(self, bucket_name: str, gcs_path: str):
        """
        Copy full directory from GCS bucket to local environment.
        :param gcs_path: Name of directory on GCS bucket.
        """
        blob_path = str(self.dir.resolve())
        blob = self._get_gcs_blob(bucket_name=bucket_name, blob_path=blob_path)
        for gcs_file in glob.glob(gcs_path + "/**", recursive=True):
            if os.path.isfile(gcs_file):
                blob.download_to_filename(blob_path)
        rich_print(
            f":cloud: :folder: Directory '{gcs_path}' from bucket '{blob.bucket.id}' downloaded to '{blob_path}' :folder: :cloud:"
        )

    def upload_directory_to_gcs(self, bucket_name: str, gcs_path: str):
        """
        Upload full base directory to GCS bucket.
        :param gcs_path: Name of directory on GCS bucket.
        """
        blob = self._get_gcs_blob(bucket_name=bucket_name, blob_path=gcs_path)
        for local_path in glob.glob(str(self.dir) + "/**", recursive=True):
            if os.path.isfile(local_path):
                blob.upload_from_filename(local_path)
        rich_print(
            f":cloud: :folder: Directory '{self.dir}' uploaded to '{gcs_path}' in bucket {blob.bucket.id} :folder: :cloud:"
        )

    def _get_gcs_blob(self, bucket_name: str, blob_path: str) -> storage.Blob:
        """Create blob that interacts with Google Cloud Storage (GCS)"""
        client = storage.Client()
        # https://console.cloud.google.com/storage/browser/[bucket_name]
        bucket = client.get_bucket(bucket_name)
        blob = bucket.blob(blob_path)
        return blob

    def _append_folder(self, folder: str) -> Path:
        """
        Return base directory Path appended with 'folder'.
        Create directory if it does not exist.
        """
        dir = Path(self.dir / folder)
        dir.mkdir(parents=True, exist_ok=True)
        return dir

    def _create_directory(self):
        """Create base directory if it does not exist."""
        if not self.dir.is_dir():
            rich_print(
                f"No existing directory found at '[blue]{self.dir}[/blue]'. Creating directory..."
            )
            self.dir.mkdir(parents=True, exist_ok=True)

    @property
    def get_all_files(self) -> list:
        """Return all contents in directory."""
        return list(self.dir.iterdir())

    @property
    def is_empty(self) -> bool:
        """Check if directory is empty."""
        return not bool(self.get_all_files)

<IPython.core.display.Javascript object>

## 1. BaseDownloader

`BaseDownloader` is an object which implements logic common to all downloaders.

To implement a new Downloader, you should inherit from `BaseDownloader` and be sure to implement at least methods `download_training_data` and `download_inference_data`.

In [ ]:
# export
@typechecked
class BaseDownloader(BaseIO):
    """
    Abstract base class for downloaders.
    :param directory_path: Base folder to download files to.
    """

    def __init__(self, directory_path: str):
        super(BaseDownloader, self).__init__(directory_path=directory_path)

    @abstractmethod
    def download_training_data(self, *args, **kwargs):
        """Download all necessary files needed for training."""
        ...

    @abstractmethod
    def download_inference_data(self, *args, **kwargs):
        """Download minimal amount of files needed for weekly inference."""
        ...

    @staticmethod
    def _load_json(file_path: str, verbose=False, *args, **kwargs) -> dict:
        """Load JSON from file and return as dictionary."""
        with open(file_path) as json_file:
            json_data = json.load(json_file, *args, **kwargs)
        if verbose:
            rich_print(json_data)
        return json_data

    def __call__(self, *args, **kwargs):
        """
        The most common use case will be to get weekly inference data. So calling the class itself returns inference data.
        """
        self.download_inference_data(*args, **kwargs)

<IPython.core.display.Javascript object>

## 2. Numerai Classic

In [ ]:
# export
class NumeraiClassicDownloader(BaseDownloader):
    """
    -WARNING- Version 1 (legacy) is deprecated. Only supporting version 2+.
    Downloading from NumerAPI for Numerai Classic data.

    :param directory_path: Base folder to download files to.
    All *args, **kwargs will be passed to NumerAPI initialization.
    """

    def __init__(self, directory_path: str, *args, **kwargs):
        super(NumeraiClassicDownloader, self).__init__(directory_path=directory_path)
        self.napi = NumerAPI(*args, **kwargs)
        self.current_round = self.napi.get_current_round()
        # NumerAPI filenames corresponding to version, class and data type
        self.version_mapping = {
            2: {
                "train": {
                    "int8": [
                        "numerai_training_data_int8.parquet",
                        "numerai_validation_data_int8.parquet"
                    ],
                    "float": [
                        "numerai_training_data.parquet",
                        "numerai_validation_data.parquet"
                    ],
                },
                "inference": {
                    "int8": ["numerai_tournament_data_int8.parquet"],
                    "float": ["numerai_tournament_data.parquet"]
                },
                "live": {
                    "int8": ['numerai_live_data_int8.parquet'],
                    "float": ['numerai_live_data.parquet']
                },
                "example": [
                    "example_predictions.parquet",
                    "example_validation_predictions.parquet",
                ],
            },
        }

    def download_training_data(
        self, subfolder: str = "", version: int = 2, int8: bool = False
    ):
        """
        Get Numerai classic training and validation data.
        :param subfolder: Specify folder to create folder within directory root. Saves in directory root by default.
        :param version: Numerai dataset version (2=super massive dataset (parquet))
        :param int8: Integer version of data
        """
        dir = self._append_folder(subfolder)
        data_type = "int8" if int8 else "float"
        train_val_files = self._get_version_mapping(version)["train"][data_type]
        for file in train_val_files:
            self.download_single_dataset(
                filename=file, dest_path=str(dir.joinpath(file))
            )

    def download_inference_data(
        self,
        subfolder: str = "",
        version: int = 2,
        int8: bool = False,
        round_num: int = None,
    ):
        """
        Get Numerai classic inference data.
        :param subfolder: Specify folder to create folder within directory root. Saves in directory root by default.
        :param version: Numerai dataset version (2=super massive dataset (parquet))
        :param int8: Integer version of data
        :param round_num: Numerai tournament round number. Downloads latest round by default.
        """
        dir = self._append_folder(subfolder)
        data_type = "int8" if int8 else "float"
        inference_files = self._get_version_mapping(version)["inference"][data_type]
        for file in inference_files:
            self.download_single_dataset(
                filename=file, dest_path=str(dir.joinpath(file)), round_num=round_num
            )

    def download_single_dataset(
        self, filename: str, dest_path: str, round_num: int = None
    ):
        """
        Download one of the available datasets through NumerAPI.

        :param filename: Name as listed in NumerAPI (Check NumerAPI().list_datasets())
        :param dest_path: Full path where file will be saved.
        :param round_num: Numerai tournament round number. Downloads latest round by default.
        """
        assert (
            filename in self.napi.list_datasets()
        ), f"Dataset '{filename}' not available in NumerAPI. Available datasets are {self.napi.list_datasets()}."
        rich_print(
            f":file_folder: [green]Downloading[/green] '{filename}' :file_folder:"
        )
        self.napi.download_dataset(
            filename=filename, dest_path=dest_path, round_num=round_num
        )

    def download_live_data(
            self,
            subfolder: str = "",
            version: int = 2,
            int8: bool = False,
            round_num: int = None
    ):
        """
        Download all live data in specified folder for given version.

        :param subfolder: Specify folder to create folder within directory root. Saves in directory root by default.
        :param version: Numerai dataset version (2=super massive dataset (parquet))
        :param int8: Integer version of data
        :param round_num: Numerai tournament round number. Downloads latest round by default.
        """
        dir = self._append_folder(subfolder)
        data_type = "int8" if int8 else "float"
        live_files = self._get_version_mapping(version)["live"][data_type]
        for file in live_files:
            self.download_single_dataset(
                filename=file, dest_path=str(dir.joinpath(file)), round_num=round_num
            )

    def download_example_data(
        self, subfolder: str = "", version: int = 2, round_num: int = None
    ):
        """
        Download all example prediction data in specified folder for given version.

        :param subfolder: Specify folder to create folder within directory root. Saves in directory root by default.
        :param version: Numerai dataset version (2=super massive dataset (parquet))
        :param round_num: Numerai tournament round number. Downloads latest round by default.
        """
        dir = self._append_folder(subfolder)
        example_files = self._get_version_mapping(version)["example"]
        for file in example_files:
            self.download_single_dataset(
                filename=file, dest_path=str(dir.joinpath(file)), round_num=round_num
            )

    def get_classic_features(self, subfolder: str = "", *args, **kwargs) -> dict:
        """
        Download feature overview (stats and feature sets) through NumerAPI and load.
        :param subfolder: Specify folder to create folder within directory root. Saves in directory root by default.
        *args, **kwargs will be passed to the JSON loader.
        """
        dir = self._append_folder(subfolder)
        filename = "features.json"
        dest_path = str(dir.joinpath(filename))
        self.download_single_dataset(filename=filename, dest_path=dest_path)
        json_data = self._load_json(dest_path, *args, **kwargs)
        return json_data

    def _get_version_mapping(self, version: int) -> dict:
        """Check if version is supported and return file mapping for version."""
        try:
            mapping_dictionary = self.version_mapping[version]
        except KeyError:
            raise NotImplementedError(
                f"Version '{version}' is not available. Available versions are {list(self.version_mapping.keys())}"
            )
        return mapping_dictionary

<IPython.core.display.Javascript object>

### BaseDownloader and NumeraiClassicDownlaoder  tests

In [ ]:
# slow
test_dir_classic = "test_numclassic_general"
numer_classic_downloader = NumeraiClassicDownloader(test_dir_classic)

# Test building class
assert isinstance(numer_classic_downloader.dir, PosixPath)
assert numer_classic_downloader.dir.is_dir()

# Test is_empty
(numer_classic_downloader.dir / "test.txt").write_text("test")
rich_print(f"Directory contents:\n{numer_classic_downloader.get_all_files}")
assert not numer_classic_downloader.is_empty

# Downloading example data
numer_classic_downloader.download_example_data("test/", round_num=290)

# Features
feature_stats_test = numer_classic_downloader.get_classic_features()
assert isinstance(feature_stats_test, dict)
assert len(feature_stats_test["feature_sets"]["legacy"]) == 304

# Remove contents
numer_classic_downloader.remove_base_directory()
assert not os.path.exists(test_dir_classic)

Directory contents:
[Path('test_numclassic_general/test1'), Path('test_numclassic_general/test.txt')]

📁 Downloading 'example_predictions.parquet' 📁

2022-01-14 12:02:26,417 INFO numerapi.utils: starting download
test_numclassic_general/test/example_predictions.parquet: 33.5MB [00:06, 5.39MB/s]                            


📁 Downloading 'example_validation_predictions.parquet' 📁

2022-01-14 12:02:34,309 INFO numerapi.utils: starting download
test_numclassic_general/test/example_validation_predictions.parquet: 13.0MB [00:03, 3.88MB/s]                            


📁 Downloading 'features.json' 📁

2022-01-14 12:02:40,672 INFO numerapi.utils: starting download
test_numclassic_general/features.json: 441kB [00:01, 351kB/s]                             


⚠ Deleting directory for 'NumeraiClassicDownloader' ⚠
Path: 
'/Users/clepelaars/Desktop/crowdcent/repositories/numerai-blocks/nbs/test_numclassic_general'

<IPython.core.display.Javascript object>

### Example usage

#### Training

In [ ]:
# slow
# Initialization
train_base_directory = "test_numclassic_train"
numer_classic_downloader = NumeraiClassicDownloader(train_base_directory)

# Uncomment line below to download training and validation data.
# numer_classic_downloader.download_training_data("train_val", int8=False)

# Get feature overview
numer_classic_downloader.get_classic_features();

No existing directory found at 'test_numclassic_train'. Creating directory...

📁 Downloading 'features.json' 📁

2022-01-14 12:02:44,765 INFO numerapi.utils: starting download
test_numclassic_train/features.json: 441kB [00:01, 247kB/s]                            


<IPython.core.display.Javascript object>

In [ ]:
# hide
# Remove contents (for tests)
numer_classic_downloader.remove_base_directory()

⚠ Deleting directory for 'NumeraiClassicDownloader' ⚠
Path: 
'/Users/clepelaars/Desktop/crowdcent/repositories/numerai-blocks/nbs/test_numclassic_train'

<IPython.core.display.Javascript object>

__For the training example the directory structure will be:__

In [ ]:
# hide_input
console = Console(record=True, width=100)

tree = Tree(
    f":file_folder: {train_base_directory} (base_directory)",
    guide_style="bold bright_black",
)
folder_tree = tree.add(":page_facing_up: features.json")
train_val_tree = tree.add(":file_folder: train_val")
train_val_tree.add(":page_facing_up: numerai_training_data.parquet")
train_val_tree.add(":page_facing_up: numerai_validation_data.parquet")

console.print(tree)

📁 test_numclassic_train (base_directory)                                                           
┣━━ 📄 features.json                                                                                
┗━━ 📁 train_val                                                                                    
    ┣━━ 📄 numerai_training_data.parquet                                                            
    ┗━━ 📄 numerai_validation_data.parquet

<IPython.core.display.Javascript object>

#### Inference

In [ ]:
# slow
# initialization
inference_base_directory = "test_numclassic_inference"
numer_classic_downloader = NumeraiClassicDownloader(inference_base_directory)

# Download tournament (inference) data
numer_classic_downloader.download_inference_data("inference", int8=False)

# Remove folder when done with inference
numer_classic_downloader.remove_base_directory()

No existing directory found at 'test_numclassic_inference'. Creating directory...

📁 Downloading inference data for round '298'.

📁 Downloading 'numerai_tournament_data.parquet' 📁

2022-01-14 12:02:49,757 INFO numerapi.utils: starting download
test_numclassic_inference/inference/numerai_tournament_data.parquet: 582MB [03:03, 3.17MB/s]                             


⚠ Deleting directory for 'NumeraiClassicDownloader' ⚠
Path: '/Users/clepelaars/Desktop/crowdcent/repositories/numerai-blocks/nbs/test_numclassic_in
ference'

<IPython.core.display.Javascript object>

__For the inference example the directory structure will be:__

In [ ]:
# hide_input
console = Console(record=True, width=100)

tree = Tree(
    f":file_folder: {inference_base_directory} (base_directory)",
    guide_style="bold bright_black",
)
inference_tree = tree.add(":file_folder: inference")
inference_tree.add(":page_facing_up: numerai_tournament_data.parquet")

console.print(tree)

📁 test_numclassic_inference (base_directory)                                                       
┗━━ 📁 inference                                                                                    
    ┗━━ 📄 numerai_tournament_data.parquet

<IPython.core.display.Javascript object>

## 3. Yahoo Finance

Use [opensignals](https://github.com/councilofelders/opensignals) to download Yahoo Finance data?

TODO Implement Yahoo Finance Downloading functionality.

In [ ]:
#export
class YahooFinanceDownloader(BaseDownloader):
    """
    Download Yahoo Finance data through opensignals.
    https://github.com/councilofelders/opensignals

    :param directory_path: Base folder to download files to.
    """

    def __init__(self, directory_path: str, *args, **kwargs):
        super(YahooFinanceDownloader, self).__init__(directory_path=directory_path)
        self.api = SignalsAPI()
        self.universe_url = self.api.TICKER_UNIVERSE_URL

    def download_inference_data(self, *args, **kwargs):
        """(minimal) weekly inference downloading here."""
        last_friday = int(
            str((datetime.now() + relativedelta(weekday=FR(-1))).date()).replace(
                "-", ""
            )
        )
        # yahoo.download_data(self.dir)

    def download_training_data(self, *args, **kwargs):
        """Training dataset downloading here."""
        ...

<IPython.core.display.Javascript object>

In [ ]:
yfinance_downloader = YahooFinanceDownloader("test_yahoo")
print(f"Universe CSV path: {yfinance_downloader.universe_url}")

# Remove folder (when done with inference)
yfinance_downloader.remove_base_directory()

No existing directory found at 'test_yahoo'. Creating directory...

Universe CSV path: https://numerai-signals-public-data.s3-us-west-2.amazonaws.com/latest_universe.csv


⚠ Deleting directory for 'YahooFinanceDownloader' ⚠
Path: '/Users/clepelaars/Desktop/crowdcent/repositories/numerai-blocks/nbs/test_yahoo'

<IPython.core.display.Javascript object>

## 4. FinnHub

<IPython.core.display.Javascript object>

## 5. Custom Downloader

We invite the Numerai Community to implement new downloaders using interesting APIs. This is especially important for creating innovative Numerai Signals models.

A new Downloader can be created by inheriting from `BaseDownloader`. You should implement methods for `download_inference_data` and `download_training_data` so every downloader has a common interface.

In [ ]:
# export
class AwesomeCustomDownloader(BaseDownloader):
    """
    - TEMPLATE -
    Download awesome financial data from who knows where.

    :param directory_path: Base folder to download files to.
    """

    def __init__(self, directory_path: str, *args, **kwargs):
        super(AwesomeCustomDownloader, self).__init__(directory_path=directory_path)

    def download_inference_data(self, *args, **kwargs):
        """(minimal) weekly inference downloading here."""
        ...

    def download_training_data(self, *args, **kwargs):
        """Training + validation dataset downloading here."""
        ...

<IPython.core.display.Javascript object>

------------------------------------------------------------

In [ ]:
# hide
# Run this cell to sync all changes with library
from nbdev.export import notebook2script

notebook2script()

Converted 00_download.ipynb.
Converted 01_dataloaders.ipynb.
Converted 02_dataset.ipynb.
Converted 03_preprocessing.ipynb.
Converted 04_model.ipynb.
Converted 05_postprocessing.ipynb.
Converted 06_modelpipeline.ipynb.
Converted 07_evaluation.ipynb.
Converted 08_key.ipynb.
Converted 09_submission.ipynb.
Converted 10_staker.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>